# Interactive Visualizations - POI-VIIRS Analysis

This notebook creates comprehensive interactive visualizations for the POI-VIIRS analysis results using Folium maps and Plotly dashboards.

## Objectives
1. Create interactive maps with POI locations and luminosity data
2. Generate comprehensive Plotly dashboards
3. Visualize spatial patterns, clusters, and anomalies
4. Create category-specific visualizations
5. Export interactive outputs for presentation


In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import json
from IPython.display import HTML, display, IFrame

# Import custom visualization modules
from visualization.interactive_maps import InteractiveMapVisualizer
from visualization.plotly_dashboard import PlotlyDashboard

# Configure plotting
%matplotlib inline

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print("Ready to create interactive visualizations...")

## 1. Initialize Visualization Components

Let's set up our visualization tools and load the integrated dataset.

In [ ]:
# Initialize visualization components
map_visualizer = InteractiveMapVisualizer(data_dir="../data", output_dir="../results")
dashboard_creator = PlotlyDashboard(data_dir="../data", output_dir="../results")

print("Visualization components initialized:")
print(f"  Map visualizer data directory: {map_visualizer.data_dir}")
print(f"  Map visualizer output directory: {map_visualizer.output_dir}")
print(f"  Dashboard creator data directory: {dashboard_creator.data_dir}")
print(f"  Dashboard creator output directory: {dashboard_creator.output_dir}")

In [ ]:
# Load the integrated dataset for preview
print("Loading integrated dataset for preview...")
poi_gdf, viirs_array = map_visualizer.load_data()

print(f"Loaded data summary:")
print(f"  POI records: {len(poi_gdf):,}")
print(f"  Categories: {poi_gdf['category_group'].nunique()}")
print(f"  Has luminosity data: {'viirs_luminosity' in poi_gdf.columns}")
print(f"  Has cluster data: {'cluster' in poi_gdf.columns}")
print(f"  Has anomaly data: {'anomaly_type' in poi_gdf.columns}")
print(f"  VIIRS raster loaded: {viirs_array is not None}")

# Display basic statistics
if 'viirs_luminosity' in poi_gdf.columns:
    print(f"\nLuminosity statistics:")
    print(f"  Range: {poi_gdf['viirs_luminosity'].min():.2f} to {poi_gdf['viirs_luminosity'].max():.2f}")
    print(f"  Mean: {poi_gdf['viirs_luminosity'].mean():.2f}")
    print(f"  Zero values: {(poi_gdf['viirs_luminosity'] == 0).sum()} ({(poi_gdf['viirs_luminosity'] == 0).mean()*100:.1f}%)")

# Show preview of data
print(f"\nData preview:")
display(poi_gdf[['name', 'category_group', 'viirs_luminosity', 'cluster', 'anomaly_type']].head() if all(col in poi_gdf.columns for col in ['viirs_luminosity', 'cluster', 'anomaly_type']) else poi_gdf.head(3))

## 2. Create Interactive Maps

Let's create comprehensive interactive maps using Folium.

In [ ]:
# Create comprehensive interactive map
print("Creating comprehensive interactive map...")
comprehensive_map_path = map_visualizer.create_comprehensive_map()

print(f"\nComprehensive map created successfully!")
print(f"File path: {comprehensive_map_path}")

# Display the map in notebook (if file exists)
if Path(comprehensive_map_path).exists():
    print(f"\nDisplaying comprehensive interactive map:")
    # Note: In a real Jupyter environment, this would display the map
    # For demonstration, we'll show the file path
    display(HTML(f'<p><strong>Interactive Map Available:</strong> <a href="{comprehensive_map_path}" target="_blank">Open Comprehensive Map</a></p>'))
    
    # You can uncomment the following line in a real Jupyter environment:
    # display(IFrame(src=comprehensive_map_path, width=1000, height=600))
else:
    print("⚠️ Map file not found. Please check the file path.")

In [ ]:
# Create category-specific maps
print("Creating category-specific interactive maps...")
category_map_paths = map_visualizer.create_category_specific_maps()

print(f"\nCategory-specific maps created: {len(category_map_paths)}")

# Display links to each category map
print(f"\nCategory-specific maps:")
for i, map_path in enumerate(category_map_paths, 1):
    map_name = Path(map_path).stem.replace('hadapsar_', '').replace('_map', '').replace('_', ' ').title()
    print(f"  {i}. {map_name}: {map_path}")
    
    # Create clickable links
    display(HTML(f'<p>{i}. <strong>{map_name}:</strong> <a href="{map_path}" target="_blank">Open {map_name} Map</a></p>'))

if len(category_map_paths) > 0:
    print(f"\n✅ Successfully created {len(category_map_paths)} category-specific maps!")
else:
    print("⚠️ No category-specific maps were created. Please check the data.")

In [ ]:
# Create temporal comparison map (demo)
print("Creating temporal comparison map (demonstration)...")
temporal_map_path = map_visualizer.create_temporal_comparison_map()

print(f"\nTemporal comparison map created:")
print(f"File path: {temporal_map_path}")

display(HTML(f'<p><strong>Temporal Analysis Map:</strong> <a href="{temporal_map_path}" target="_blank">Open Temporal Comparison Map</a></p>'))

print(f"\nNote: This is a demonstration map using current data.")
print(f"For true temporal analysis, you would need multiple years of data.")

## 3. Create Interactive Dashboards

Now let's create comprehensive interactive dashboards using Plotly.

In [ ]:
# Create all interactive dashboards
print("Creating comprehensive interactive dashboards...")
dashboard_paths = dashboard_creator.create_all_dashboards()

print(f"\nInteractive dashboards created: {len(dashboard_paths)}")

# Display each dashboard
print(f"\nDashboard Summary:")
print(f"=" * 50)

for dashboard_type, file_path in dashboard_paths.items():
    if file_path:  # Only display if file was created successfully
        dashboard_title = dashboard_type.replace('_', ' ').title()
        print(f"\n📊 {dashboard_title} Dashboard:")
        print(f"   File: {file_path}")
        
        # Create description based on dashboard type
        descriptions = {
            'overview': 'Comprehensive overview with multiple analysis components',
            'spatial': 'Focused on spatial distribution and geographic patterns',
            'correlation': 'Analysis of POI-luminosity relationships and correlations',
            'cluster': 'Detailed cluster analysis and spatial groupings'
        }
        
        description = descriptions.get(dashboard_type, 'Interactive analysis dashboard')
        print(f"   Description: {description}")
        
        # Create clickable link
        display(HTML(f'<div style="margin: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">'
                    f'<h4>📊 {dashboard_title} Dashboard</h4>'
                    f'<p>{description}</p>'
                    f'<a href="{file_path}" target="_blank" style="background: #007bff; color: white; padding: 5px 10px; text-decoration: none; border-radius: 3px;">Open Dashboard</a>'
                    f'</div>'))

if len(dashboard_paths) > 0:
    print(f"\n✅ Successfully created {len(dashboard_paths)} interactive dashboards!")
else:
    print("⚠️ No dashboards were created. Please check the data and try again.")

## 4. Create Quick Preview Visualizations

Let's create some quick preview visualizations that can be displayed inline in the notebook.

In [ ]:
# Create quick preview visualizations using Plotly (inline)
print("Creating quick preview visualizations...")

# 1. Interactive scatter plot of POI locations
if len(poi_gdf) > 0:
    fig_scatter = px.scatter_mapbox(
        poi_gdf,
        lat=poi_gdf.geometry.y,
        lon=poi_gdf.geometry.x,
        color='category_group',
        size='viirs_luminosity' if 'viirs_luminosity' in poi_gdf.columns else None,
        hover_name='name',
        hover_data={'category_group': True, 'viirs_luminosity': True} if 'viirs_luminosity' in poi_gdf.columns else {'category_group': True},
        mapbox_style="open-street-map",
        title="Interactive POI Distribution - Hadapsar, Pune",
        height=600,
        zoom=14,
        center=dict(lat=18.5050, lon=73.9350)
    )
    
    fig_scatter.update_layout(margin=dict(r=0, t=50, l=0, b=0))
    fig_scatter.show()
    
    print("✅ Interactive scatter plot created and displayed above")
else:
    print("⚠️ No POI data available for scatter plot")

In [ ]:
# 2. Interactive luminosity distribution
if 'viirs_luminosity' in poi_gdf.columns:
    fig_hist = px.histogram(
        poi_gdf,
        x='viirs_luminosity',
        color='category_group',
        title="VIIRS Luminosity Distribution by Category",
        labels={'viirs_luminosity': 'VIIRS Luminosity', 'count': 'Number of POIs'},
        marginal='rug',
        height=500
    )
    
    fig_hist.update_layout(bargap=0.1)
    fig_hist.show()
    
    print("✅ Interactive histogram created and displayed above")
else:
    print("⚠️ No luminosity data available for histogram")

In [ ]:
# 3. Interactive box plot for category analysis
if 'viirs_luminosity' in poi_gdf.columns:
    fig_box = px.box(
        poi_gdf,
        x='category_group',
        y='viirs_luminosity',
        title="Luminosity Distribution by POI Category",
        labels={'category_group': 'POI Category', 'viirs_luminosity': 'VIIRS Luminosity'},
        height=500
    )
    
    fig_box.update_xaxes(tickangle=45)
    fig_box.update_layout(margin=dict(b=100))
    fig_box.show()
    
    print("✅ Interactive box plot created and displayed above")
else:
    print("⚠️ No luminosity data available for box plot")

In [ ]:
# 4. Interactive cluster visualization (if available)
if 'cluster' in poi_gdf.columns and 'viirs_luminosity' in poi_gdf.columns:
    fig_cluster = px.scatter(
        poi_gdf,
        x=poi_gdf.geometry.x,
        y='viirs_luminosity',
        color='cluster',
        size='viirs_luminosity',
        hover_name='name',
        hover_data={'category_group': True, 'anomaly_type': True} if 'anomaly_type' in poi_gdf.columns else {'category_group': True},
        title="Cluster Analysis: Longitude vs Luminosity",
        labels={'x': 'Longitude', 'viirs_luminosity': 'VIIRS Luminosity', 'cluster': 'Cluster ID'},
        height=500
    )
    
    fig_cluster.update_traces(marker=dict(line=dict(width=1, color='white')))
    fig_cluster.show()
    
    print("✅ Interactive cluster visualization created and displayed above")
elif 'cluster' in poi_gdf.columns:
    # Simple cluster scatter without luminosity
    fig_cluster = px.scatter(
        poi_gdf,
        x=poi_gdf.geometry.x,
        y=poi_gdf.geometry.y,
        color='cluster',
        hover_name='name',
        hover_data={'category_group': True},
        title="Spatial Cluster Distribution",
        labels={'x': 'Longitude', 'y': 'Latitude', 'cluster': 'Cluster ID'},
        height=500
    )
    
    fig_cluster.show()
    print("✅ Simple cluster visualization created and displayed above")
else:
    print("⚠️ No cluster data available for cluster visualization")

## 5. Visualization Summary and Export

Let's create a summary of all visualizations created and provide export information.

In [ ]:
# Create visualization summary
print("VISUALIZATION SUMMARY")
print("=" * 60)

# Count successful visualizations
total_maps = 1 + len(category_map_paths) + 1  # comprehensive + category + temporal
total_dashboards = len([path for path in dashboard_paths.values() if path])
total_previews = 4  # The inline visualizations we created above

print(f"\n📊 Visualization Statistics:")
print(f"   Interactive Maps Created: {total_maps}")
print(f"   Interactive Dashboards Created: {total_dashboards}")
print(f"   Inline Preview Plots: {total_previews}")
print(f"   Total Visualizations: {total_maps + total_dashboards + total_previews}")

# List all output files
print(f"\n📁 Output Files:")
print(f"\nInteractive Maps:")
print(f"  1. Comprehensive Map: {comprehensive_map_path}")
for i, map_path in enumerate(category_map_paths, 2):
    map_name = Path(map_path).stem.replace('hadapsar_', '').replace('_map', '').title()
    print(f"  {i}. {map_name}: {map_path}")
print(f"  {len(category_map_paths) + 2}. Temporal Comparison: {temporal_map_path}")

print(f"\nInteractive Dashboards:")
for i, (dashboard_type, file_path) in enumerate(dashboard_paths.items(), 1):
    if file_path:
        dashboard_title = dashboard_type.replace('_', ' ').title()
        print(f"  {i}. {dashboard_title}: {file_path}")

# Create index file for easy access
print(f"\n📋 Creating visualization index...")

index_html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Hadapsar POI-VIIRS Analysis - Visualization Index</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }}
        h1 {{ color: #333; border-bottom: 3px solid #007bff; padding-bottom: 10px; }}
        h2 {{ color: #007bff; margin-top: 30px; }}
        .section {{ margin: 20px 0; padding: 15px; background: #f8f9fa; border-radius: 5px; }}
        .viz-link {{ display: inline-block; margin: 5px 10px 5px 0; padding: 8px 15px; 
                    background: #007bff; color: white; text-decoration: none; border-radius: 3px; }}
        .viz-link:hover {{ background: #0056b3; }}
        .stats {{ background: #e7f3ff; padding: 15px; border-left: 4px solid #007bff; margin: 20px 0; }}
    </style>
</head>
<body>
    <h1>🗺️ Hadapsar POI-VIIRS Analysis - Interactive Visualizations</h1>
    
    <div class="stats">
        <h3>📊 Summary Statistics</h3>
        <ul>
            <li>Total POIs Analyzed: {len(poi_gdf):,}</li>
            <li>Categories: {poi_gdf['category_group'].nunique()}</li>
            <li>Interactive Maps: {total_maps}</li>
            <li>Interactive Dashboards: {total_dashboards}</li>
        </ul>
    </div>
    
    <div class="section">
        <h2>🗺️ Interactive Maps</h2>
        <p>Explore spatial patterns and relationships through interactive Folium maps:</p>
        <a href="{Path(comprehensive_map_path).name}" class="viz-link" target="_blank">📍 Comprehensive Map</a>
"""

# Add category maps
for map_path in category_map_paths:
    map_name = Path(map_path).stem.replace('hadapsar_', '').replace('_map', '').replace('_', ' ').title()
    index_html += f'        <a href="{Path(map_path).name}" class="viz-link" target="_blank">🏢 {map_name}</a>\n'

index_html += f'        <a href="{Path(temporal_map_path).name}" class="viz-link" target="_blank">⏰ Temporal Analysis</a>\n'

# Add dashboards section
index_html += f"""
    </div>
    
    <div class="section">
        <h2>📊 Interactive Dashboards</h2>
        <p>Comprehensive analytical dashboards with multiple visualization components:</p>
"""

for dashboard_type, file_path in dashboard_paths.items():
    if file_path:
        dashboard_title = dashboard_type.replace('_', ' ').title()
        index_html += f'        <a href="{Path(file_path).name}" class="viz-link" target="_blank">📈 {dashboard_title} Dashboard</a>\n'

index_html += f"""
    </div>
    
    <div class="section">
        <h2>📋 Analysis Notes</h2>
        <ul>
            <li><strong>Study Area:</strong> Hadapsar, Pune, India</li>
            <li><strong>Data Sources:</strong> Foursquare POI data + VIIRS Nighttime Lights</li>
            <li><strong>Analysis Type:</strong> Spatial integration and correlation analysis</li>
            <li><strong>Generated:</strong> {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}</li>
        </ul>
    </div>
    
    <div class="section">
        <h2>🔍 How to Use</h2>
        <ol>
            <li><strong>Interactive Maps:</strong> Click on POI markers for details, use layer controls to toggle views</li>
            <li><strong>Dashboards:</strong> Hover over charts for details, use zoom and pan controls</li>
            <li><strong>Analysis:</strong> Look for patterns in POI-luminosity relationships and spatial clusters</li>
        </ol>
    </div>
</body>
</html>
"""

# Save index file
index_path = map_visualizer.output_dir / "maps" / "index.html"
with open(index_path, 'w', encoding='utf-8') as f:
    f.write(index_html)

print(f"📋 Visualization index created: {index_path}")

# Display the index link
display(HTML(f'<div style="margin: 20px 0; padding: 20px; background: #e7f3ff; border-radius: 10px; text-align: center;">'
            f'<h3>🎉 All Visualizations Complete!</h3>'
            f'<p>Access all interactive visualizations through the index page:</p>'
            f'<a href="{index_path}" target="_blank" style="background: #28a745; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; font-weight: bold;">📋 Open Visualization Index</a>'
            f'</div>'))

print(f"\n✅ All visualizations completed successfully!")
print(f"📁 All files are saved in: {map_visualizer.output_dir}")
print(f"🌐 Open the index.html file to access all visualizations")

## 6. Visualization Analysis Insights

Let's provide some guidance on interpreting the visualizations we've created.

In [ ]:
# Provide visualization interpretation guidance
print("VISUALIZATION INTERPRETATION GUIDE")
print("=" * 60)

interpretation_guide = {
    "Interactive Maps": {
        "Purpose": "Explore spatial relationships and geographic patterns",
        "Key Features": [
            "POI locations colored by category or luminosity",
            "Clustering and anomaly overlays",
            "Luminosity heatmap layers",
            "Category-specific views"
        ],
        "What to Look For": [
            "Spatial clustering of similar POI types",
            "Areas with high luminosity but few commercial POIs",
            "Commercial areas with unexpectedly low luminosity",
            "Patterns in POI distribution across the study area"
        ]
    },
    "Interactive Dashboards": {
        "Purpose": "Comprehensive analytical exploration with multiple linked views",
        "Key Features": [
            "Multi-panel layouts with coordinated views",
            "Statistical summaries and correlations",
            "Cluster analysis results",
            "Anomaly detection outcomes"
        ],
        "What to Look For": [
            "Strong correlations between categories and luminosity",
            "Cluster quality and separation",
            "Distribution patterns and outliers",
            "Category-specific luminosity characteristics"
        ]
    },
    "Spatial Patterns": {
        "Purpose": "Understand urban development and infrastructure patterns",
        "Indicators": {
            "High Luminosity + Commercial POIs": "Well-developed commercial areas",
            "High Luminosity + Non-Commercial POIs": "Well-lit residential/institutional areas",
            "Low Luminosity + Commercial POIs": "Potential infrastructure gaps",
            "Low Luminosity + Essential Services": "Areas needing lighting improvement"
        }
    },
    "Urban Planning Insights": {
        "Purpose": "Inform urban development and infrastructure decisions",
        "Applications": [
            "Identify areas needing improved street lighting",
            "Plan commercial development in well-lit areas",
            "Ensure essential services have adequate lighting",
            "Balance residential and commercial development"
        ]
    }
}

for section, content in interpretation_guide.items():
    print(f"\n{section.upper()}")
    print("-" * len(section))
    
    if "Purpose" in content:
        print(f"Purpose: {content['Purpose']}")
    
    for key, value in content.items():
        if key == "Purpose":
            continue
        elif isinstance(value, list):
            print(f"\n{key}:")
            for item in value:
                print(f"  • {item}")
        elif isinstance(value, dict):
            print(f"\n{key}:")
            for sub_key, sub_value in value.items():
                print(f"  • {sub_key}: {sub_value}")

print(f"\n" + "=" * 60)
print("RECOMMENDED ANALYSIS WORKFLOW")
print("=" * 60)

workflow_steps = [
    "1. Start with the Comprehensive Map to get an overall view",
    "2. Use the Overview Dashboard to understand key statistics",
    "3. Explore Category-specific maps to understand POI distribution",
    "4. Examine the Correlation Dashboard for POI-luminosity relationships",
    "5. Use the Cluster Dashboard to understand spatial groupings",
    "6. Investigate anomalies using the Spatial Dashboard",
    "7. Compare findings across different visualization types",
    "8. Document insights for urban planning applications"
]

for step in workflow_steps:
    print(f"  {step}")

print(f"\n📈 Ready for analysis! All visualizations are now available.")

## 7. Final Summary

Complete summary of the visualization workflow and outputs.

In [ ]:
# Final comprehensive summary
print("🎊 VISUALIZATION WORKFLOW COMPLETE! 🎊")
print("=" * 70)

# Create final summary
final_summary = {
    "Project": "Hadapsar POI-VIIRS Analysis - Interactive Visualizations",
    "Study Area": "Hadapsar, Pune, India",
    "Data Sources": ["Foursquare POI Data", "VIIRS Nighttime Lights"],
    "Analysis Methods": ["Spatial Integration", "Clustering Analysis", "Anomaly Detection", "Correlation Analysis"],
    "Visualizations Created": {
        "Interactive Maps": total_maps,
        "Interactive Dashboards": total_dashboards,
        "Inline Previews": total_previews,
        "Total": total_maps + total_dashboards + total_previews
    },
    "Key Features": [
        "Comprehensive spatial mapping",
        "Category-specific analysis",
        "Interactive data exploration",
        "Statistical dashboards",
        "Anomaly identification",
        "Cluster visualization"
    ],
    "Output Formats": ["HTML Interactive Maps", "HTML Interactive Dashboards", "Jupyter Notebook Plots"],
    "Applications": [
        "Urban planning and development",
        "Infrastructure assessment",
        "Economic activity analysis",
        "Public safety planning",
        "Resource allocation optimization"
    ]
}

print(f"\n📋 PROJECT SUMMARY:")
for key, value in final_summary.items():
    if isinstance(value, dict):
        print(f"\n{key}:")
        for sub_key, sub_value in value.items():
            print(f"  {sub_key}: {sub_value}")
    elif isinstance(value, list):
        print(f"\n{key}:")
        for item in value:
            print(f"  • {item}")
    else:
        print(f"\n{key}: {value}")

print(f"\n" + "=" * 70)
print("🎯 NEXT STEPS & RECOMMENDATIONS:")
print("=" * 70)

recommendations = [
    "🗺️ Share interactive maps with urban planners and stakeholders",
    "📊 Present dashboard findings to decision-makers",
    "🔍 Investigate anomalous areas identified in the analysis",
    "💡 Use insights for infrastructure improvement planning",
    "📈 Consider expanding analysis to other Pune neighborhoods",
    "🔄 Repeat analysis with updated data for temporal comparison",
    "📝 Document methodology for replication in other cities",
    "🤝 Collaborate with local authorities on implementation"
]

for rec in recommendations:
    print(f"  {rec}")

print(f"\n" + "=" * 70)
print("📁 ACCESS YOUR VISUALIZATIONS:")
print(f"Main Index: {index_path}")
print(f"Maps Directory: {map_visualizer.output_dir / 'maps'}")
print(f"Dashboards Directory: {dashboard_creator.output_dir / 'dashboards'}")
print("=" * 70)

print(f"\n🎉 SUCCESS! Your POI-VIIRS analysis visualizations are ready for exploration!")
print(f"📧 Share the index.html file with colleagues for easy access to all visualizations.")
print(f"🌟 Great work on completing this comprehensive urban analysis project!")